##### Example: Fitting to Cluster 0

In [ ]:
params = {
    'batch_size': 512,
    'num_epochs': 10,
    'lr': 0.001,
    'embedding_dim': 64,
    'dropout': 0.3,
    'alpha': 0.01,  # Decay constant for days since interaction (days)
    'beta': 0.01    # Decay constant for video age (days)
}

cluster = 0
train_cluster = train_processed[train_processed['cluster'] == cluster]

cluster, cluster_0_predictions = train_and_predict(params, train_cluster, **{'cluster': 0})

----- Training for cluster 0 -----
Initialising...
Model moved to cpu
Epoch [1/10], Loss: 0.4294
Epoch [2/10], Loss: 0.3775
Epoch [3/10], Loss: 0.3646
Epoch [4/10], Loss: 0.3584
Epoch [5/10], Loss: 0.3541
Epoch [6/10], Loss: 0.3518
Epoch [7/10], Loss: 0.3496
Epoch [8/10], Loss: 0.3471
Epoch [9/10], Loss: 0.3445
Epoch [10/10], Loss: 0.3416
Generating predictions...
Complete!


In [ ]:
cluster_0_predictions

,user_id,video_id,watch_ratio
0,14,103,1.407040
1,21,103,1.994147
2,24,103,0.806597
3,51,103,1.448327
4,64,103,2.313566
...,...,...,...
634835,7086,10130,1.066982
634836,7116,10130,1.063808
634837,7132,10130,1.176456
634838,7141,10130,1.193652


### Hyperparameter tuning

In [ ]:
# hyperparameters = {
#     'batch_size': [256, 512],
#     'num_epochs': [10],
#     'lr': [0.001, 0.01],
#     'embedding_dim': [32, 64],
#     'dropout': [0.3, 0.5],
#     'alpha': [0.01, 0.05],
#     'beta': [0.01, 0.05]
# }
hyperparameters = {
    'batch_size': [128],
    'num_epochs': [10],
    'lr': [0.001, 0.005],
    'embedding_dim': [64],
    'dropout': [0.3],
    'alpha': [0.01, 0.05],
    'beta': [0.01, 0.05]
}

# Generate all possible combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))

# Train for each combination of hyperparameters
for params in param_combinations:
    params_dict = {key: val for key, val in zip(hyperparameters.keys(), params)}
    print(f"Training with hyperparameters: {params_dict}")
    train_by_cluster_and_without(params_dict, train_processed, False, True)

Training with hyperparameters: {'batch_size': 128, 'num_epochs': 10, 'lr': 0.001, 'embedding_dim': 64, 'dropout': 0.3, 'alpha': 0.01, 'beta': 0.01}
----- Training for cluster 0 -----
Initialising...
Model moved to cpu
Epoch [1/10], Loss: 0.4035
Epoch [2/10], Loss: 0.3653
Epoch [3/10], Loss: 0.3577
Epoch [4/10], Loss: 0.3539
Epoch [5/10], Loss: 0.3512
Epoch [6/10], Loss: 0.3472
Epoch [7/10], Loss: 0.3432
Epoch [8/10], Loss: 0.3379
Epoch [9/10], Loss: 0.3320
Epoch [10/10], Loss: 0.3250
Generating predictions...
Complete!
Predictions for cluster 0 saved to ../KuaiRec 2.0/results/cluster0_batch_size128_num_epochs10_lr0.001_embedding_dim64_dropout0.3_alpha0.01_beta0.01.csv
----- Training for cluster 1 -----
Initialising...
Model moved to cpu
Epoch [1/10], Loss: 0.2081
Epoch [2/10], Loss: 0.1805
Epoch [3/10], Loss: 0.1773
Epoch [4/10], Loss: 0.1753
Epoch [5/10], Loss: 0.1740
Epoch [6/10], Loss: 0.1724
Epoch [7/10], Loss: 0.1705
Epoch [8/10], Loss: 0.1682
Epoch [9/10], Loss: 0.1656
Epoch [10/

In [ ]:
for param_str, metrics_df in metrics_dict.items():
    print(f"Metrics for model with hyperparameters: {param_str}")
    print(metrics_df)
    metrics_df.to_csv(root + f'results/metrics_{param_str}.csv', index=False)

Metrics for model with hyperparameters: batch_size128_num_epochs10_lr0.001_embedding_dim64_dropout0.3_alpha0.01_beta0.01
   cluster   NDCG@50  Distinct Categories @ 50  Avg Watch Ratio @ 50  \
0        0  0.861192                 19.795539                   NaN   
0        1  0.864238                 20.229117                   NaN   
0        2  0.859480                 19.918841                   NaN   
0        3  0.862338                 20.087302                   NaN   
0  Overall  0.861985                 20.032601                   NaN   

   Avg Precision@500  Avg Recall@500  Avg F1@500  
0           0.409159        0.890557    0.550859  
0           0.442618        0.883331    0.581756  
0           0.414749        0.896581    0.559391  
0           0.426409        0.907213    0.571986  
0           0.425083        0.894346    0.567780  
Metrics for model with hyperparameters: batch_size128_num_epochs10_lr0.001_embedding_dim64_dropout0.3_alpha0.01_beta0.05
   cluster   NDCG@5

In [ ]:
hyperparameters = {
    'batch_size': [512],
    'num_epochs': [10],
    'lr': [0.001],
    'embedding_dim': [64],
    'dropout': [0.3],
    'decay': [0.01]
}

metrics_dict = {}

# Generate all possible combinations of hyperparameters
param_combinations = list(itertools.product(*hyperparameters.values()))
for params in param_combinations:
    params_dict = {key: val for key, val in zip(hyperparameters.keys(), params)}
    
    print(f"Getting metrics for model with hyperparameters: {params_dict}")
    param_str = '_'.join([f'{key}{val}' for key, val in params_dict.items()])
    prediction_scores = pd.read_csv(root + f'results/w_clustering_{param_str}.csv')

    recommendations = get_user_recommendations(prediction_scores, videos_in_val_data, user_watch_history)

    metrics_df = get_all_metrics(k1, k2, ground_truth, recommendations, video_data, threshold, by_cluster=True)
    
    print(metrics_df)
    metrics_df.to_csv(root + f'results/metrics_w_clustering_{param_str}.csv', index=False)

    metrics_dict[param_str] = metrics_df

Getting metrics for model with hyperparameters: {'batch_size': 128, 'num_epochs': 10, 'lr': 0.001, 'embedding_dim': 64, 'dropout': 0.3, 'alpha': 0.01, 'beta': 0.01}


100%|██████████| 378/378 [06:26<00:00,  1.02s/it]


   cluster   NDCG@50  Distinct Categories @ 50  Avg Watch Ratio @ 50  \
0        0  0.866925                 19.475836              0.851770   
0        1  0.872250                 20.968974              0.843245   
0        2  0.878964                 19.991304              0.827713   
0        3  0.890984                 20.473545              0.843225   
0  Overall  0.877895                 20.312544              0.841067   

   Avg Precision@500  Avg Recall@500  Avg F1@500  \
0           0.411122        0.893490    0.553181   
0           0.443067        0.883498    0.582142   
0           0.422958        0.912366    0.570056   
0           0.427160        0.908164    0.572850   
0           0.427799        0.899069    0.571176   

   Avg Predicted Watch Ratio @ 50  
0                        2.767446  
0                        5.633058  
0                        5.297689  
0                        4.917240  
0                        4.812979  
Getting metrics for model with hyperpa

100%|██████████| 378/378 [04:46<00:00,  1.32it/s]


   cluster   NDCG@50  Distinct Categories @ 50  Avg Watch Ratio @ 50  \
0        0  0.890414                 19.182156              0.851770   
0        1  0.882869                 20.470167              0.843245   
0        2  0.879713                 18.944928              0.827713   
0        3  0.888588                 19.330688              0.843225   
0  Overall  0.885068                 19.546421              0.841067   

   Avg Precision@500  Avg Recall@500  Avg F1@500  \
0           0.407048        0.886358    0.548099   
0           0.439373        0.877667    0.577685   
0           0.413537        0.894245    0.557824   
0           0.424007        0.902681    0.568872   
0           0.422777        0.890078    0.564827   

   Avg Predicted Watch Ratio @ 50  
0                        7.469201  
0                       21.365817  
0                       16.723155  
0                       15.359612  
0                       15.972299  
Getting metrics for model with hyperpa

...

   cluster   NDCG@50  Distinct Categories @ 50  Avg Watch Ratio @ 50  \
0        0  0.875152                 19.479554              0.851770   
0        1  0.900511                 18.995227              0.843245   
0        2  0.899111                 18.344928              0.827713   
0        3  0.889532                 19.333333              0.843225   
0  Overall  0.892393                 19.019135              0.841067   

   Avg Precision@500  Avg Recall@500  Avg F1@500  \
0           0.407813        0.887664    0.549047   
0           0.442637        0.882891    0.581646   
0           0.414958        0.896834    0.559624   
0           0.426055        0.906150    0.571438   
0           0.424788        0.893441    0.567312   

   Avg Predicted Watch Ratio @ 50  
0                      450.991987  
0                      532.518502  
0                      530.215591  
0                      702.617710  
0                      561.981552  
Getting metrics for model with hyperpa

100%|██████████| 378/378 [00:50<00:00,  7.48it/s]


   cluster   NDCG@50  Distinct Categories @ 50  Avg Watch Ratio @ 50  \
0        0  0.893854                 18.579926              0.851770   
0        1  0.904882                 18.424821              0.843245   
0        2  0.895255                 18.342029              0.827713   
0        3  0.891960                 18.626984              0.843225   
0  Overall  0.896964                 18.488306              0.841067   

   Avg Precision@500  Avg Recall@500  Avg F1@500  \
0           0.408222        0.888538    0.549602   
0           0.444509        0.886030    0.583941   
0           0.414697        0.896508    0.559330   
0           0.426261        0.906765    0.571752   
0           0.425413        0.894625    0.568112   

   Avg Predicted Watch Ratio @ 50  
0                      583.661448  
0                      814.858474  
0                      854.518418  
0                     1177.563561  
0                      877.646003  
Getting metrics for model with hyperpa

100%|██████████| 378/378 [00:50<00:00,  7.50it/s]

   cluster   NDCG@50  Distinct Categories @ 50  Avg Watch Ratio @ 50  \
0        0  0.869848                 19.847584              0.851770   
0        1  0.860227                 20.427208              0.843245   
0        2  0.858149                 19.272464              0.827713   
0        3  0.873621                 19.433862              0.843225   
0  Overall  0.865141                 19.768249              0.841067   

   Avg Precision@500  Avg Recall@500  Avg F1@500  \
0           0.407932        0.887662    0.549155   
0           0.440408        0.879223    0.578932   
0           0.414082        0.895198    0.558500   
0           0.425002        0.904214    0.570093   
0           0.423653        0.891433    0.565892   

   Avg Predicted Watch Ratio @ 50  
0                     1309.364564  
0                     3178.485867  
0                     3054.243347  
0                     2595.693403  
0                     2635.641890  


In [ ]:
for param_str, metrics_df in metrics_dict.items():
    print(f"Metrics for model with hyperparameters: {param_str}")
    print(metrics_df)

Metrics for model with hyperparameters: batch_size128_num_epochs10_lr0.001_embedding_dim64_dropout0.3_alpha0.01_beta0.01
   cluster   NDCG@50  Distinct Categories @ 50  Avg Watch Ratio @ 50  \
0        0  0.866925                 19.475836              0.851770   
0        1  0.872250                 20.968974              0.843245   
0        2  0.878964                 19.991304              0.827713   
0        3  0.890984                 20.473545              0.843225   
0  Overall  0.877895                 20.312544              0.841067   

   Avg Precision@500  Avg Recall@500  Avg F1@500  \
0           0.411122        0.893490    0.553181   
0           0.443067        0.883498    0.582142   
0           0.422958        0.912366    0.570056   
0           0.427160        0.908164    0.572850   
0           0.427799        0.899069    0.571176   

   Avg Predicted Watch Ratio @ 50  
0                        2.767446  
0                        5.633058  
0                        

### Jia weis

In [ ]:
# With segmentation
prediction_scores = pd.read_csv(root + f'results/w_clustering_batch_size512_num_epochs20_lr0_001_embedding_dim64.csv')

recommendations = get_user_recommendations(prediction_scores, videos_in_val_data, user_watch_history)

test1 = get_all_metrics(k1, k2, ground_truth, recommendations, video_data, threshold, by_cluster=True)
test1

100%|██████████| 378/378 [00:49<00:00,  7.57it/s]


,cluster,NDCG@50,Distinct Categories @ 50,Avg Watch Ratio @ 50,Avg Precision@500,Avg Recall@500,Avg F1@500,Avg Predicted Watch Ratio @ 50
0,0,0.869462,19.710037,0.851770,0.415694,0.902736,0.559226,2.478849
0,1,0.879717,19.878282,0.843245,0.453711,0.903085,0.595794,4.193733
0,2,0.875481,20.014493,0.827713,0.424419,0.915455,0.572021,3.965236
0,3,0.873250,20.534392,0.843225,0.435600,0.924449,0.583820,3.574282
0,Overall,0.874994,20.055280,0.841067,0.434449,0.911766,0.579802,3.644982


In [ ]:
# Without segmentation
prediction_scores = pd.read_csv(root + f'results/wo_clustering_batch_size512_num_epochs20_lr0_001_embedding_dim64.csv')

recommendations = get_user_recommendations(prediction_scores, videos_in_val_data, user_watch_history)

test2 = get_all_metrics(k1, k2, ground_truth, recommendations, video_data, threshold, by_cluster=False)
test2

100%|██████████| 1411/1411 [03:09<00:00,  7.45it/s]


,cluster,NDCG@50,Distinct Categories @ 50,Avg Watch Ratio @ 50,Avg Precision@500,Avg Recall@500,Avg F1@500,Avg Predicted Watch Ratio @ 50
0,Overall,0.904384,19.932672,0.841067,0.437712,0.92446,0.585115,8.400009
